## Getting top bollywood movies form IMDB using webscraping

In [1]:
# Use beautifulSoup to get the latest data

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

In [3]:
# UDF to generate urls

def url_gen(a):
#     a simple miner assist tool to generate URLs 
#     iterater from range(1, n) where n == R
    return 'https://www.imdb.com/search/title/?countries=in&locations=India&count=250&start=' \
                + str( (251) + (a * 250) ) \
                + '&ref_=adv_nxt'

In [4]:
# define empty lists to fetch scraped data from the miner
bollywood_title_all = []
bollywood_year_all = []
bollywood_title_links_all = []

In [5]:
def miner(url = 'https://www.imdb.com/search/title/?countries=in&locations=India&count=250'
          , bollywood_title_all = bollywood_title_all
          , bollywood_year_all = bollywood_year_all
          , bollywood_title_links_all = bollywood_title_links_all):

    # The miner takes in a default URL which is the 1st page to be mined.
    # for subsequent mines, we will use the url_gen function to pass in new urls.
    # the miner takes in the empty lists by default and updates them through each iteration.
    
    # Downloading imdb top 250 movie's data
    url = url
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')

    # use select method and enter the selector path of the element
    bollywood_title = (soup.select('div.lister-item-content > h3 > a'))
    bollywood_year = (soup.select('div.lister-item-content > h3 > span.lister-item-year.text-muted.unbold'))
    bollywood_title_links = [a.attrs.get('href') for a in soup.select('div.lister-item-content > h3 > a')]

    # convert bs4.element into string
    for i in range(0, len(bollywood_title)):
        bollywood_title[i] = bollywood_title[i].text

    for i in range(0, len(bollywood_year)):
        bollywood_year[i] = bollywood_year[i].text

    # Update the empty lists with new mined data in this iteration
    bollywood_title_all += bollywood_title
    bollywood_year_all += bollywood_year
    bollywood_title_links_all += bollywood_title_links 

In [6]:
try:
    # call the miner with default page
    miner()
    print('mined no.', 1, 'page' )

    # iterate over the next i pages pages:
    for i in range(1, 50):
        miner(url = url_gen(i))
        print('mined no.', i + 1, 'page' )

except: 
    print("MINING ERROR !! DATA NOT MINED")

mined no. 1 page
mined no. 2 page
mined no. 3 page
mined no. 4 page
mined no. 5 page
mined no. 6 page
mined no. 7 page
mined no. 8 page
mined no. 9 page
mined no. 10 page
mined no. 11 page
mined no. 12 page
mined no. 13 page
mined no. 14 page
mined no. 15 page
mined no. 16 page
mined no. 17 page
mined no. 18 page
mined no. 19 page
mined no. 20 page
mined no. 21 page
mined no. 22 page
mined no. 23 page
mined no. 24 page
mined no. 25 page
mined no. 26 page
mined no. 27 page
mined no. 28 page
mined no. 29 page
mined no. 30 page
mined no. 31 page
mined no. 32 page
mined no. 33 page
mined no. 34 page
mined no. 35 page
mined no. 36 page
mined no. 37 page
mined no. 38 page
mined no. 39 page
mined no. 40 page
mined no. 41 page
mined no. 42 page
mined no. 43 page
mined no. 44 page
mined no. 45 page
mined no. 46 page
mined no. 47 page
mined no. 48 page
mined no. 49 page
mined no. 50 page


In [7]:
dict_for_df = {"title":bollywood_title_all, "release_year":bollywood_year_all, "titleId":bollywood_title_links_all}

In [8]:
scraped_bollywood = pd.DataFrame(dict_for_df)
scraped_bollywood.release_year = scraped_bollywood.release_year.str.extract('(\d+)')
scraped_bollywood.titleId = scraped_bollywood.titleId.str.extract('(\d+)')

In [9]:
scraped_bollywood

,title,release_year,titleId
0,Shershaah,2021,10295212
1,Mimi,2021,10895576
2,#Home,2021,10534500
3,Boomika,2021,11073148
4,The Family Man,2019,9544034
...,...,...,...
12495,Anand,1971,0066763
12496,Baadshaho,2017,4906960
12497,I,2015,2302966
12498,Iruvar,1997,0119385


In [10]:
scraped_bollywood.describe().T

,count,unique,top,freq
title,12500,9448,Drishyam,24
release_year,12148,98,2018,1004
titleId,12500,9750,6588966,12


In [11]:
title_basics_raw = pd.read_csv("compressed_tsv/title.basics.tsv.gz" , sep='\t', low_memory = False)

In [12]:
title_basics_raw.tconst = title_basics_raw.tconst.str.extract('(\d+)')
title_basics_raw

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
...,...,...,...,...,...,...,...,...,...
8230930,9916848,tvEpisode,Episode #3.17,Episode #3.17,0,2010,\N,\N,"Action,Drama,Family"
8230931,9916850,tvEpisode,Episode #3.19,Episode #3.19,0,2010,\N,\N,"Action,Drama,Family"
8230932,9916852,tvEpisode,Episode #3.20,Episode #3.20,0,2010,\N,\N,"Action,Drama,Family"
8230933,9916856,short,The Wind,The Wind,0,2015,\N,27,Short


In [13]:
if title_basics_raw[title_basics_raw.tconst == '9614490'].shape >(1,1):
    print("Successfully mined and reconciled")
else:
    print("FAILED to reconcile")

Successfully mined and reconciled


In [14]:
!ls

IMDB Data Explorer.ipynb compressed_tsv
IMDB Data Miner.ipynb    mined_csv


In [15]:
# save the df as a csv on local machine
scraped_bollywood.to_csv('mined_csv/scraped_bollywood.csv')

In [16]:
print("saved csv to local drive")

saved csv to local drive
